In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
prds = pd.read_csv('./archive (1)/products.csv')
aisles= pd.read_csv('./archive (1)/aisles.csv')
dpts = pd.read_csv('./archive (1)/departments.csv')

In [4]:
prds.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [5]:
dpts.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [9]:
prod_desc = pd.merge(pd.merge(prds,aisles, on = 'aisle_id',how ='inner'),dpts,on = 'department_id',how= 'inner')

In [10]:
prod_desc.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [12]:
prod_desc['metadata']= prod_desc.apply(lambda x:x['aisle']+' '+x['department']+' '+ x['product_name'],axis =1 )

In [13]:
prod_desc.head()

,product_id,product_name,aisle_id,department_id,aisle,department,metadata
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,cookies cakes snacks Chocolate Sandwich Cookies
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks,cookies cakes snacks Nutter Butter Cookie Bite...
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks,cookies cakes snacks Danish Butter Cookies
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks,cookies cakes snacks Gluten Free All Natural C...
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks,cookies cakes snacks Mini Nilla Wafers Munch Pack


In [17]:
count_vec = CountVectorizer(stop_words = 'english')
count_vec_matrix = count_vec.fit_transform(prod_desc['metadata'])

In [18]:
count_vec_matrix.shape

(49688, 10624)

In [34]:
# Function definition : This function takes in words and vectorizes them, then finds similar vector in the count_vec_matrix
def metadata_search_engine(product_input):

    vec = count_vec.transform(pd.Series(product_input))
    cosine_sim = cosine_similarity(vec, count_vec_matrix)
    similarity_score = pd.DataFrame(cosine_sim.reshape(49688,), index = prod_desc.index, columns=['score'])
    non_zero_scores = similarity_score[similarity_score['score'] > 0]

    if len(non_zero_scores) == 0:
        print('No similar products found.  Please refine your search terms and try again')
        return

    if len(non_zero_scores) < 10:
        item_count = len(non_zero_scores)
    else:
        item_count = 10

    similarity_scores = similarity_score.sort_values(['score'], ascending=False)[:item_count]

    return prod_desc['product_name'].iloc[similarity_scores.index]

In [36]:
metadata_search_engine('Salads')

46461             Tabouli
46427               Chili
46452             Edamame
46475        Lentil Salad
46482        Potato Salad
46501           Miso Soup
46496    The Vegan Caesar
46410       Caprese Salad
46490        Caesar Salad
46488       Zucchini Soup
Name: product_name, dtype: object

In [37]:
metadata_search_engine('Shampoo')

34053                Shampoo Baby Shampoo
24763                    Everyday Shampoo
43535                  Strawberry Shampoo
24147                Lice Killing Shampoo
34099                      Shampoo & Wash
22935                             Shampoo
31653    Carpet & Rug Shampoo Concentrate
23281      2 in 1 Shampoo and Conditioner
23181                    Therappe Shampoo
22757                  Nourishing Shampoo
Name: product_name, dtype: object

In [38]:
metadata_search_engine('Bath')

34151                     Bedtime Bath™ Baby Bath
34145                         Soothing Vapor Bath
34088                    Bubble Bath Extra Gentle
34168                 Eucalyptus Mint Bubble Bath
34085           Over Tired and Cranky Bubble Bath
34090                           Bedtime Baby Bath
49623                         Cellular Detox Bath
34059          Everyday Shea Lavender Bubble Bath
34170    Super Sensitive No Fragrance Bubble Bath
34105                           Baby Bedtime Bath
Name: product_name, dtype: object